In [7]:
import pandas as pd
import numpy as np
import re
from openpyxl import load_workbook
from openpyxl.styles import PatternFill


In [8]:
df = pd.read_excel('tuition_fees_comp_ai.xlsx')
df.head()

,university,faculty,program,campus,tuition_fee_text,tuition_fee_numeric,more_info_url
0,จุฬาลงกรณ์มหาวิทยาลัย,คณะวิศวกรรมศาสตร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,วิทยาเขตหลัก,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...,25500.0,https://www.reg.chula.ac.th/th/information/fee/
1,จุฬาลงกรณ์มหาวิทยาลัย,คณะวิศวกรรมศาสตร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,วิทยาเขตหลัก,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...,25500.0,https://www.reg.chula.ac.th/th/information/fee/
2,จุฬาลงกรณ์มหาวิทยาลัย,คณะวิศวกรรมศาสตร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมหุ...,วิทยาเขตหลัก,ดูรายละเอียดเพิ่มเติม https://www.chula.ac.th/...,NaN,https://www.chula.ac.th/academics/admissions/t...
3,มหาวิทยาลัยเกษตรศาสตร์,คณะวิศวกรรมศาสตร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,บางเขน,0,NaN,NaN
4,มหาวิทยาลัยเกษตรศาสตร์,คณะวิศวกรรมศาสตร์ กำแพงแสน,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,กำแพงแสน,"156,000",NaN,NaN


In [5]:
# Function to clean and extract per-semester tuition fee
def extract_per_semester_fee(row):
    text = str(row['tuition_fee_text']).strip()
    numeric_fee = row['tuition_fee_numeric']
    
    # Handle special case: "เทอมแรก X บาท เทอมต่อไปเทอมละ Y บาท"
    if 'เทอมแรก' in text and 'เทอมต่อไปเทอมละ' in text:
        try:
            # Extract the fee for subsequent semesters (after the first semester)
            number = float(''.join(filter(str.isdigit, text.split('เทอมต่อไปเทอมละ')[1].split('บาท')[0].replace(',', ''))))
            return number
        except:
            return np.nan
    
    # Handle cases with per-semester keywords
    if any(keyword in text for keyword in ['ต่อภาคการศึกษา', 'ต่อภาคเรียน', 'บาท/เทอม', 'อัตราค่าเล่าเรียน']):
        try:
            # Prioritize per-semester fee if available
            if 'หรือ' in text and 'ตลอดหลักสูตร' in text:
                text = text.split('หรือ')[0]  # Take only the part before "หรือ"
            elif 'ตลอดหลักสูตร' in text and '(' in text and 'ต่อภาค' in text:
                # Handle cases like "100000 ตลอดหลักสูตร (14000 ต่อภาคเรียน)"
                text = text.split('(')[1].split(')')[0]  # Extract content in parentheses
            # Extract number before "บาท"
            number = float(''.join(filter(str.isdigit, text.split('บาท')[0].replace(',', ''))))
            return number
        except:
            return np.nan
    
    # If numeric_fee is provided and not 0, check if it needs conversion
    if pd.notna(numeric_fee) and numeric_fee != 0:
        # If numeric_fee is likely a per-semester fee (less than 100,000), use it
        if numeric_fee < 100000:
            return numeric_fee
        # If numeric_fee is likely a total program fee (100,000 or more), convert it
        try:
            semesters = 8  # Assume 4-year program with 2 semesters per year
            return numeric_fee / semesters
        except:
            return np.nan
    
    # Handle cases where tuition is for the entire program
    if 'ตลอดหลักสูตร' in text:
        try:
            # Extract total program fee
            total_fee = float(''.join(filter(str.isdigit, text.split('บาท')[0].replace(',', ''))))
            # Assume 4-year program with 2 semesters per year (8 semesters total)
            semesters = 8
            return total_fee / semesters
        except:
            return np.nan
    
    # Return NaN for empty or unclear cases
    return np.nan

In [6]:
# Create new column for per-semester tuition fee
df['tuition_fee_numeric'] = df.apply(extract_per_semester_fee, axis=1)

# Save the cleaned data to a new Excel file
output_file = 'cleaned_tuition_fees_updated.xlsx'
df.to_excel(output_file, index=False)

# Apply yellow highlight to rows where tuition_fee_numeric is NaN or 0
wb = load_workbook(output_file)
ws = wb.active
yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

# Find the column index for 'tuition_fee_numeric'
header = [cell.value for cell in ws[1]]
col_idx = header.index('tuition_fee_numeric') + 1  # 1-based indexing in openpyxl

# Highlight rows where tuition_fee_numeric is NaN or 0
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col_idx, max_col=col_idx):
    cell = row[0]
    if pd.isna(cell.value) or cell.value == 0:
        for col in ws.iter_cols(min_row=cell.row, max_row=cell.row, min_col=1, max_col=ws.max_column):
            for c in col:
                c.fill = yellow_fill

wb.save(output_file)